In [1]:
import rlcms.composites as composites
import ee
import geemap

ee.Initialize()

In [2]:
region = ee.Geometry.Polygon([[-120.70162384826843,36.658181801020476],
                               [-120.65218537170593,36.658181801020476],
                               [-120.65218537170593,36.68461734040718],
                               [-120.70162384826843,36.68461734040718],
                               [-120.70162384826843,36.658181801020476]])
composite = composites.composite(dataset='Sentinel2',
                                 aoi=region,
                                 start_date='2020-01-01',
                                 end_date='2022-12-31',
                                 indices=['IBI','SAVI'],
                                 composite_mode='seasonal',
                                 season=[1,2,3,4,5],
                                 reducer=ee.Reducer.percentile([10,50,99]), # 'mean', 'min'
                                 addJRC=True,
                                 addTopography=True,
                                 addHarmonics=True,
                                 harmonicsOptions={'nir':{'start':1,'end':180}})



In [3]:
Map = geemap.Map()
vis_params = {'bands':['t0_red_p50','t0_blue_p50','t0_green_p50'],
              'min':0,'max':1500,
            #   'palette':['red','green','blue']
              }
Map.addLayer(region,{},'test rect')
Map.addLayer(composite,vis_params,'composite')
Map.centerObject(region,12)

Map

Map(center=[36.671401368184334, -120.67690460998617], controls=(WidgetControl(options=['position', 'transparen…